In [31]:
using WTP
using SCDM
using Test
using LinearAlgebra

Read the wave functions from the `wfc?.dat` files.

In [32]:
const test_1_dir = "../test/test_data/test_1"
wave_functions_list = wave_functions_from_directory(joinpath(test_1_dir, "si.save"));

In [33]:
u = wannier_from_save(wave_functions_list)
k_map, brillouin_zone = i_kpoint_map(wave_functions_list);
# wave_functions_list = nothing # release the memory

## Notations

### Grid Indexing

Indexing a grid yields a grid vector. 
For example, indexing the Brillouin zone yields a kpoint.

In [34]:
Γ = brillouin_zone[0, 0, 0]

GridVector{BrillouinZone3D}:
    _coefficients: [0, 0, 0]


### Band Indexing

Indexing the object `u` with a kpoint and a band index gives an orbital.

In [35]:
u_1_Γ = u[Γ][1]

ket
grid:
    ReciprocalLattice3D((ket: -0.612, -0.612, 0.612, ket: 0.612, 0.612, 0.612, ket: -0.612, 0.612, -0.612), ((-12, 11), (-12, 11), (-12, 11)))
kpoint:
    GridVector{BrillouinZone3D}:
        _coefficients: [0, 0, 0]
    
band:
    1

### Orbital Indexing

An orbital can be indexed by a grid vector, which can be obtained by indexing a grid.

In [36]:
reciprocal_lattice = grid(u_1_Γ)

type: ReciprocalLattice3D
domain: ((-12, 11), (-12, 11), (-12, 11))
basis:
    ket: -0.612, -0.612, 0.612
    ket: 0.612, 0.612, 0.612
    ket: -0.612, 0.612, -0.612

In [37]:
orbital_norm = 0
for G in reciprocal_lattice
    orbital_norm += abs2(u_1_Γ[G])
end
orbital_norm

0.9999999999999976

### Perform a Fourier transform.

In [38]:
u_real = ifft(u);

### Perform an SCDM.

In [78]:
U = scdm_condense_phase(u_real, collect(1:20));

LoadError: MethodError: objects of type Complex{Bool} are not callable
Maybe you forgot to use an operator such as [36m*, ^, %, / etc. [39m?

In [68]:
U[Γ]

20×20 Matrix{ComplexF64}:
   0.220397-0.0469504im   …    0.0940595-0.0146979im
   0.207291-0.299289im         0.0187225-0.0402761im
  -0.268417+0.0992527im       -0.0200048-0.00244999im
  -0.169643-0.0780606im       0.00436412+0.0138434im
  0.0997161+0.025793im        -0.0107589-0.000567389im
  -0.121138+0.0476662im   …    0.0157134+0.000340948im
  0.0322895-0.00553159im     -0.00513031+0.012819im
   0.226064-0.0975009im        -0.206777+0.105184im
  0.0378104+0.175016im        -0.0180337-0.0966849im
  -0.138616-0.190889im         0.0163735+0.0189412im
  -0.212651-0.0591912im   …     0.273434+0.0827288im
  0.0516555-0.0067475im       -0.0117035-0.0428774im
  0.0053448-0.0101693im       -0.0695488-0.127674im
 -0.0133018+0.0181965im        0.0307237+0.00625571im
  -0.104749-0.0830201im        -0.639294-0.636184im
 -0.0434948+0.0133361im   …    0.0322762-0.0252773im
   0.183678+0.0515426im        0.0176984-0.0263148im
  -0.127523-0.250956im         0.0195777-0.0126257im
 -0.0320437-0.1327

Verify the result by comparing it to the output of Anil's Matlab code.
We read the `amn` file produced by the Matlab code and contruct a gauge.

In [69]:
amn = AMN(joinpath(test_1_dir, "unk", "si.amn"))
U_matlab = Gauge(brillouin_zone, amn, k_map, false);

In [76]:
for k in brillouin_zone
    @test norm(U[k] - U_matlab[k]) < 1e-5
end

In [19]:
U_matlab[Γ]

20×20 Matrix{ComplexF64}:
   0.220397-0.0469504im   …    0.0940595-0.0146979im
   0.207291-0.299289im         0.0187225-0.0402761im
  -0.268417+0.0992527im       -0.0200048-0.00244999im
  -0.169643-0.0780606im       0.00436412+0.0138434im
  0.0997161+0.025793im        -0.0107589-0.000567389im
  -0.121138+0.0476662im   …    0.0157134+0.000340948im
  0.0322895-0.00553159im     -0.00513031+0.012819im
   0.226064-0.0975009im        -0.206777+0.105184im
  0.0378104+0.175016im        -0.0180337-0.0966849im
  -0.138616-0.190889im         0.0163735+0.0189412im
  -0.212651-0.0591912im   …     0.273434+0.0827288im
  0.0516555-0.0067475im       -0.0117035-0.0428774im
  0.0053448-0.0101693im       -0.0695488-0.127674im
 -0.0133018+0.0181965im        0.0307237+0.00625571im
  -0.104749-0.0830202im        -0.639294-0.636184im
 -0.0434948+0.0133361im   …    0.0322762-0.0252773im
   0.183678+0.0515426im        0.0176984-0.0263148im
  -0.127523-0.250956im         0.0195777-0.0126257im
 -0.0320437-0.1327

In [18]:
U[Γ]

20×20 Matrix{ComplexF64}:
   0.0351608-0.18668im     …    -0.0653557+0.0536939im
   0.0717031-0.123341im           0.092391+0.00092641im
    0.182487-0.00130938im     -0.000805109-0.0557736im
   -0.300406-0.157748im         -0.0159457+0.0325468im
   0.0681666+0.137218im          -0.161034+0.0425359im
    0.160186+0.180599im    …      0.046608+0.00395483im
   0.0314086+0.117126im         -0.0167712+0.040016im
  -0.0957558-0.201156im          -0.142488-0.248178im
     0.26726+0.249331im         -0.0881065+0.0523417im
 -0.00912637-0.0157625im        0.00248623-0.0296463im
    0.166552-0.0531958im   …     0.0523666-0.391779im
   0.0697087+0.196419im          -0.377086-0.058293im
    0.185133+0.00480448im         0.076885+0.0351499im
  -0.0274167+0.0314689im        -0.0115886+0.0845463im
   0.0694782+0.0963492im         0.0298293+0.0255014im
    0.178538+0.36341im     …    -0.0647763+0.0458345im
  -0.0134163+0.264442im         -0.0149777+0.320285im
   0.0812385-0.27338im            -0.44881

### Centers and Spreads

Compute the centers and the spreads of the Wannier function.
The numeric values are outputs from Wannier90.

Constructing the MMN matrix is in fact a $O(N^2 N_g N_b)$ time algorithm. $N_b$ is the number of neighbors, which can be treated as a constant.

In [20]:
scheme = W90FiniteDifference(u);

Progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Time: 0:00:03


In [21]:
M = gauge_transform(neighbor_basis_integral(scheme), U);

In [22]:
@test isapprox(center(M, scheme, 1), [1.751302, -3.656156, -3.154520], atol = 1e-6)
@test isapprox(center(M, scheme, 2), [0.662933, 1.224859, 0.588340], atol = 1e-6)
@test isapprox(center(M, scheme, 3), [0.751350, -1.252297, 0.334295], atol = 1e-6)
@test isapprox(center(M, scheme, 4), [0.745342, 0.390014, -1.239850], atol = 1e-6)

Test Passed

In [23]:
spread(n) = second_moment(M, scheme, n) - norm(center(M, scheme, n))^2

@test isapprox(spread(1), 15.47179726, atol = 1e-6)
@test isapprox(spread(2), 13.17038702, atol = 1e-6)
@test isapprox(spread(3), 7.64407335, atol = 1e-6)
@test isapprox(spread(4), 8.25678268, atol = 1e-6)

Test Passed

In [ ]:
using CairoMakie

In [ ]:
scatter(1:20, spread.(1:20))